In [7]:
# !pip install langchain_community
# !pip install pypdf
# !pip install fastembed
# !pip install chromadb

In [8]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import os

In [9]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "RAG-COURSE"

In [10]:
def ingest():
    # Get the doc
    loader = PyPDFLoader("User Manual.pdf")
    pages = loader.load_and_split()
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    #
    embedding = FastEmbedEmbeddings()
    #Create vector store
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./sql_chroma_db")

#ingest() #run only first time

In [11]:
from langchain_ollama import ChatOllama

def rag_chain():
    #model = ChatOllama(model="llama3.2",temperalture=0)
    model = ChatOllama(model="qwen2.5:3b-instruct-q6_K", temperalture=0)
    prompt = PromptTemplate.from_template(
        """
        <s> [Instructions] You are a friendly assistant. Answer the question based only on the following context. 
        If you don't know the answer, then reply, No Context availabel for this question {input}. [/Instructions] </s> 
        [Instructions] Question: {input} 
        Context: {context} 
        Answer: [/Instructions]
        """
    )
    #Load vector store
    embedding = FastEmbedEmbeddings()
    vector_store = Chroma(persist_directory="./sql_chroma_db", embedding_function=embedding)

    #Create chain
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.5,
        },
    )

    document_chain = create_stuff_documents_chain(model, prompt)
    chain = create_retrieval_chain(retriever, document_chain)

    return chain

def ask(query: str):
    chain = rag_chain()
    result = chain.invoke({"input": query})
    print(result["answer"])
    for doc in result["context"]:
        print("Source: ", doc.metadata["source"])

In [ ]:
ask("how to change password")

Based on the context provided, here are the steps to change your password:

1. **Login to NBP Digital**: Access the application through its digital platform.

2. **Access Hamburger Menu and Change Password Option**: After logging in, navigate to the hamburger menu (usually represented by three horizontal lines) and select the "Change Password" option from there.

3. **Input Fields Appear**: You will be presented with input fields for:
   - Old Password
   - New Password
   - Re-enter Password

4. **Enter Information and Submit**: Provide your old password, new password, and re-enter the new password to ensure consistency. Press the submit button.

5. **Confirmation Notification**: After submitting the form, a confirmation message will display: "Dear Customer, Your Password has been changed successfully!"

It's worth noting that if you need to reset your password due to forgotten credentials or changes in email address, follow these steps:

1. **Login Page for Forgot Password**:
   - Wh